In [ ]:
ks_cols 


cs.ends_with((_ks))

In [ ]:
import polars as pl

stat_cols = pl.selectors.ends_with('_ks', '_kw', '_med')

df = (
    pl.read_csv("C:/Users/ben.hardcastle/Downloads/fit_non_paramertic_trial.csv")
    .select('unit_id', stat_cols, pl.lit('trial').alias('interval'))
    # ----------------------------------------------------------------- #
    .vstack(
        pl.read_csv("C:/Users/ben.hardcastle/Downloads/fit_non_paramertic_quiescent.csv")
        .select('unit_id', stat_cols, pl.lit('baseline').alias('interval'))
    )
    .vstack(
        pl.read_csv("C:/Users/ben.hardcastle/Downloads/fit_non_paramertic_response.csv")
        .select('unit_id', stat_cols, pl.lit('response').alias('interval'))
    )
    .unpivot(
        index=['unit_id' ,'interval'],
        variable_name='label',
    )
    .with_columns(
        pl.col('label').str.split('_').list.get(0).alias('result_type'),
        pl.col('label').str.split('_').list.get(1).str.strip_prefix('block').cast(pl.Int8).alias('block_index'),
        pl.col('label').str.split('_').list.get(-1).alias('test')
    )
    .drop('label')
    .pivot(
        on='result_type',
        values='value',
    )
    .group_by('test', 'unit_id', 'interval')
    .agg(
        pl.col('p').min().alias('p_min'),
        pl.col('D').max().alias('D_max'),
        pl.col('F').max().alias('F_max'),
    )
    .pivot(
        on=['interval'],
        values=['p_min', 'D_max', 'F_max'],
    )
)
def get_rename_map(test: str, drop: str) -> dict:
    return {
        f'{stat}_{interval}': f'{test}_{stat}_{interval}'
        for interval in ['trial', 'baseline', 'response']
        for stat in [s for s in ('p_min', 'D_max', 'F_max') if not s.startswith(drop)]
    }

for test, drop in [('ks', 'F_'), ('kw', 'D_'), ('med', 'F_')]:
    df.filter(pl.col('test') == test).drop('test', pl.selectors.starts_with(drop)).rename(get_rename_map(test, drop)).write_parquet(f"//allen/programs/mindscope/workgroups/dynamicrouting/ben/{test}_test.parquet")
df

test,unit_id,p_min_baseline,p_min_response,p_min_trial,D_max_baseline,D_max_response,D_max_trial,F_max_baseline,F_max_response,F_max_trial
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""kw""","""715710_2024-07-16_B-104""",0.015902,0.000039,0.000022,null,null,null,5.813666,16.898001,17.977588
"""ks""","""668755_2023-08-30_D-59""",0.00684,0.006274,0.015459,0.2181,0.219743,0.20167,null,null,null
"""ks""","""649943_2023-02-13_A-94""",0.007748,0.056317,0.064442,0.215035,0.172404,0.171638,null,null,null
"""kw""","""702136_2024-03-05_B-244""",0.003366,0.000091,0.000005,null,null,null,8.597515,15.324463,20.969585
"""kw""","""726088_2024-06-20_A-519""",6.3415e-39,2.9673e-39,3.4385e-39,null,null,null,170.306646,171.81683,171.523743
…,…,…,…,…,…,…,…,…,…,…
"""ks""","""666986_2023-08-17_F-416""",0.281651,0.429842,0.632149,0.125,0.108696,0.092093,null,null,null
"""med""","""681532_2023-10-18_C-369""",0.000317,0.546526,0.405322,12.964471,0.363577,0.692481,null,null,null
"""kw""","""686176_2023-12-04_E-220""",3.6336e-26,3.0413e-21,1.4486e-27,null,null,null,111.966714,89.516237,118.356576


In [32]:
def get_rename_map(test: str) -> dict:
    return {
        f'{stat}_{interval}': f'{test}_{stat}_{interval}'
        for interval in ['trial', 'baseline', 'response']
        for stat in ['p_min', 'D_max', 'F_max']
    }
get_rename_map('ks')

{'p_min_trial': 'ks_p_min_trial',
 'D_max_trial': 'ks_D_max_trial',
 'F_max_trial': 'ks_F_max_trial',
 'p_min_baseline': 'ks_p_min_baseline',
 'D_max_baseline': 'ks_D_max_baseline',
 'F_max_baseline': 'ks_F_max_baseline',
 'p_min_response': 'ks_p_min_response',
 'D_max_response': 'ks_D_max_response',
 'F_max_response': 'ks_F_max_response'}